## Run inference on set of recordings

Given a set of videos and a DLC trained model, run inference. 

In [1]:
import os
import deeplabcut
from glob import glob

/home/blansdel/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/deeplabcut/__init__.py:78: UserWarning: 
        As PyTorch is not installed, unsupervised identity learning will not be available.
        Please run `pip install torch`, or ignore this warning.
        
  warnings.warn(


In [3]:
#Path to DLC model config file
path_config_file = '/home/blansdel/projects/behavior/dlc_projects/overnight_videos/Overnight_Videos_813a_816b_combined_iter0/config.yaml'

#video_path = '/home/blansdel/ImageAnalysisScratch/AnimalBehaviorCore/Zakharenko/animal-behavior-ml/inference/Overnight\ Videos'
video_path = '/home/blansdel/ImageAnalysisScratch/AnimalBehaviorCore/Zakharenko/animal-behavior-ml/inference/Overnight\ Videos\ LSD'

video_pattern = '**/**/*cropped.mp4'
cameras_to_process = ['e3v813a', 'e3v816b'] #Only process videos from these cameras

create_video = True         # Whether or not to plot detections
shuffle = 1                 # Which shuffle (model version) to use
overwrite = False           # Whether to overwrite current 
track_method = 'ellipse'    # Which tracker to use (ellipse or box). Ellipse is the default, and generally performs better
n_animals = 1               # Number of animals to track (can be different to what the model was trained on)

Find the videos to analyze

In [ ]:
videos = sorted(glob(os.path.join(video_path, video_pattern)))
these_videos = [v for v in videos if v.split('/')[-1].split('-')[0] in cameras_to_process]

Run the analysis

In [ ]:
deeplabcut.analyze_videos(path_config_file, these_videos, videotype='.mp4', n_tracks = n_animals)
deeplabcut.stitch_tracklets(path_config_file, these_videos, videotype='mp4', shuffle=shuffle, trainingsetindex=0)

Filter predictions

In [ ]:
#Filter by directory (this seems to be a DLC bug?, in that the other functions work for multiple video dirs)
dirs = set([os.path.dirname(v) for v in these_videos])
for d in dirs:
    these_dir_vids = [v for v in these_videos if os.path.dirname(v) == d]
    deeplabcut.filterpredictions(path_config_file, these_dir_vids, save_as_csv=True)

Create video with detections and create labeled video of results, if desired

In [ ]:
if create_video:
    deeplabcut.create_video_with_all_detections(path_config_file, these_videos)
    deeplabcut.create_labeled_video(path_config_file, these_videos, videotype='.mp4', shuffle=1, trainingsetindex=0, filtered=True)